In [102]:
#PART 1
import numpy as np
import pandas as pd

In [503]:
individual = pd.read_csv("C:/Users/Orysya/Desktop/Civis_Analytics_Assessment/individual_data.csv")
household = pd.read_csv("C:/Users/Orysya/Desktop/Civis_Analytics_Assessment/household_data.csv")

In [21]:
individual.head()

,hh_id,person_id,is_head_of_household,married,gender,race,age,voted_in_2012,is_college_graduate
0,1,1,1,Single,Female,White,28,0,1
1,1,2,0,Single,Male,White,27,1,1
2,2,3,1,Married,Female,White,34,0,1
3,2,4,0,Married,Male,White,33,1,0
4,2,5,0,Single,Male,White,6,0,1


In [22]:
#1. Binary attribute: 1 for adult, 0 for child
def adult(row):
    if row['age'] < 18:
        return 0
    else:
        return 1
individual['adult'] = individual.apply(adult, axis=1)

In [23]:
#1. What percent of the sample is adults?
adult_percentage = (float(individual['adult'][individual['adult'] == 1].count()) / float(individual['adult'].count())) * 100.00
print 'Adults make up', format(adult_percentage, '.2f'),'% of the entire sample'

Adults make up 76.46 % of the entire sample


In [24]:
# 2. Average number of adults per household?
A = individual[individual['adult'] == 1].groupby('hh_id')['adult'].count() #Provides count of adults for each hh_id
B = individual['hh_id'].nunique() #Provides count of unique number of households
average_adults_per_household = float(sum(A))/B
print 'The average number of adults per household is', format(average_adults_per_household, '.2f')

The average number of adults per household is 1.61


In [25]:
# 2. Average number of children per household?
A = individual[individual['adult'] == 0].groupby('hh_id')['adult'].count() #Provides count of children for each hh_id
B = individual['hh_id'].nunique() #Provides count of unique number of households
average_children_per_household = float(sum(A))/B
print 'The average number of children per household is', format(average_children_per_household, '.2f')

The average number of children per household is 0.50


In [26]:
# 3. What percent of adults are college graduates?
A = individual[individual['adult'] == 1].groupby('is_college_graduate')['adult'].count()
B = individual['adult'][individual['adult'] == 1].count()
adults_college_graduates = (float(A[1])/float(B)) * 100.00
print 'The percent of adults who are college graduate is', format(adults_college_graduates, '.2f'), '%'

The percent of adults who are college graduate is 70.12 %


In [27]:
# 3. What percentage of people are college graduates?
collegegraduate_percentage = (float(individual['is_college_graduate'][individual['is_college_graduate'] == 1].count()) / float(individual['is_college_graduate'].count())) * 100.00
print 'The percent of people (including children) who are college graduates is', format(collegegraduate_percentage, '.2f'), '%'

The percent of people (including children) who are college graduates is 70.07 %


In [28]:
# 3. Do you find coding of this variable (college graduates) reliable?
print 'By comparing the variables adults_college_graduates and collegegradaute_percentage, the variable is_college_graduate should is not reliable since it is expected that collegegraduate_percentage < adults_college_graduates, but examination of the dataset shows that non-adults were shown to be college graduates, which does not make sense'

By comparing the variables adults_college_graduates and collegegradaute_percentage, the variable is_college_graduate should is not reliable since it is expected that collegegraduate_percentage < adults_college_graduates, but examination of the dataset shows that non-adults were shown to be college graduates, which does not make sense


In [43]:
# 4. Create a variable that indicates the type of household (hh_type): “single male head of household,” “single female head of household,” “married couple,” or “unmarried couple.” 
individual['gender'] = individual['gender'].replace({'.*[fF].*': 'Female'}, regex=True)
individual['gender'] = individual['gender'].replace({'.*[M].*': 'Male'}, regex=True)

In [44]:
# Unmarried_couple logic
A = individual[individual['married'] == 'Single'].groupby('hh_id')['married'].count() #groupby hh_id, count of single marriage status
B = individual[individual['adult'] == 1].groupby('hh_id')['adult'].count() #groupby hh_id, count of adult status
logical = ((A > 1) & (B > 1))
key = list(logical.index)
values = list(logical)
dic_uc = dict(zip(key, values))
for i, j in dic_uc.items():
    if j == True:
        dic_uc[i] = 'unmarried_couple'
    else:
        del dic_uc[i]
dic_uc
A_dic_uc = list(dic_uc.keys())
global A_dic_uc

In [45]:
# Married_couple logic
C = individual[individual['married'] == 'Married'].groupby('hh_id')['married'].count()
logical = (C > 1)
key = list(logical.index)
values = list(logical)
dic_mc = dict(zip(key, values))
for i, j in dic_mc.items():
    if j == True:
        dic_mc[i] = 'married_couple'
    else:
        del dic_mc[i]
dic_mc
B_dic_mc = list(dic_mc.keys())
global B_dic_mc

In [46]:
# Single logic - determines if the head of household is not in a relationship
B = individual[individual['adult'] == 1].groupby('hh_id')['adult'].count() #groupby hh_id, count of adult status
logical = (B == 1)
key = list(logical.index)
values = list(logical)
dic_s = dict(zip(key, values))
for i, j in dic_s.items():
    if j == True:
        dic_s[i] = 'single'
    else:
        del dic_s[i]
C_dic_s = list(dic_s.keys())
global C_dic_s

In [47]:
# Find gender for each head of household
B = individual[individual['is_head_of_household'] == 1]
key = list(B['hh_id'])
value = list(B['gender'])
dic_in = dict(zip(key, value))
global dic_in

In [48]:
# Creates new variables, hh_type describing type of household
def hh_type(row):
    global B_dic_mc #dictionary keys to indicate which households belong to married couples
    global A_dic_uc #dictionary keys to indicate which households belong to unmarried couples
    global C_dic_s #dictionary keys to indicate which households belong to single couples
    global dic_in #dictionary to indicate keys & associated gender values for each hh_id, will be passed within function
    if row['hh_id'] in B_dic_mc:
        return 'married_couple'
    elif row['hh_id'] in A_dic_uc:
        return 'unmarried_couple'
    elif row['hh_id'] in C_dic_s:
        return 'single_{0}_head_of_household'.format(dic_in[row['hh_id']])
individual['hh_type'] = individual.apply(hh_type, axis=1)
individual

,hh_id,person_id,is_head_of_household,married,gender,race,age,voted_in_2012,is_college_graduate,adult,hh_type
0,1,1,1,Single,Female,White,28,0,1,1,unmarried_couple
1,1,2,0,Single,Male,White,27,1,1,1,unmarried_couple
2,2,3,1,Married,Female,White,34,0,1,1,married_couple
3,2,4,0,Married,Male,White,33,1,0,1,married_couple
4,2,5,0,Single,Male,White,6,0,1,0,married_couple
5,3,6,1,Single,Female,White,28,0,1,1,unmarried_couple
6,3,7,0,Single,Male,White,29,1,0,1,unmarried_couple
7,4,8,1,Single,Female,White,30,0,1,1,single_Female_head_of_household
8,4,9,0,Single,Female,White,3,0,0,0,single_Female_head_of_household
9,5,10,1,Single,Female,White,30,1,1,1,single_Female_head_of_household


In [50]:
# What percent of households are of each type?
A = ((individual.groupby('hh_type').hh_id.nunique())/10000) *100.00
print 'Married couples make up', A[0], '%, single female head of households make up', A[1], '%, single male head of households make up', A[2], '%,and unmarried couples make up', A[3], '% of the total households.'

Married couples make up 24.34 %, single female head of households make up 9.56 %, single male head of households make up 29.52 %,and unmarried couples make up 36.58 % of the total households.


In [51]:
individual.to_csv("C:/Users/Orysya/Desktop/individual_editted.csv")

In [68]:
#PART 2
A = individual[['hh_id', 'hh_type', 'race']]
subset = A.drop_duplicates(subset=['hh_id', 'hh_type', 'race'])
subset.to_csv("C:/Users/Orysya/Desktop/socio_info.csv", index=False)

In [79]:
socio_economic.head()

,hh_id,hh_income,is_urban,is_owner_of_home,tercile_of_census_tract_income,hh_type,race
0,1,111000,1,0,Middle,unmarried_couple,White
1,2,123000,1,1,Bottom,married_couple,White
2,3,105400,1,0,Middle,unmarried_couple,White
3,4,48100,1,0,Top,single_Female_head_of_household,White
4,5,54600,0,0,Top,single_Female_head_of_household,White


In [95]:
socio_economic = pd.merge(household, subset, how='inner', on='hh_id')
A = socio_economic.groupby(['hh_type', 'race', 'tercile_of_census_tract_income'])
A['hh_income'].describe()

hh_type           race   tercile_of_census_tract_income       
married_couple    Asian  Bottom                          count       147.000000
                                                         mean      99455.782313
                                                         std       10178.644911
                                                         min       75000.000000
                                                         25%       92700.000000
                                                         50%       99200.000000
                                                         75%      106900.000000
                                                         max      120200.000000
                         Middle                          count       183.000000
                                                         mean      98406.557377
                                                         std        9583.660583
                                                         

In [101]:
A['is_urban'].value_counts()

hh_type                          race   tercile_of_census_tract_income   
married_couple                   Asian  Bottom                          1      87
                                                                        0      60
                                        Middle                          1     112
                                                                        0      71
                                        Top                             1     344
                                                                        0     229
                                 Black  Bottom                          1      34
                                                                        0      20
                                        Middle                          1      76
                                                                        0      46
                                        Top                             1     165
                        

In [99]:
A['is_owner_of_home'].value_counts()

hh_type                          race   tercile_of_census_tract_income   
married_couple                   Asian  Bottom                          1     147
                                        Middle                          1     183
                                        Top                             1     573
                                 Black  Bottom                          1      54
                                        Middle                          1     122
                                        Top                             1     272
                                 White  Bottom                          1     324
                                        Middle                          1     467
                                        Top                             1    1422
single_Female_head_of_household  Asian  Bottom                          0      30
                                        Middle                          0      40
                        

In [538]:
# PART 3
# Please build a model that produces an individual level prediction of a person’s probability to register as a Democrat. In Dataset C, approximately the first 10,000 individuals have known registration status while for the approximately other 10,000 individuals, registration is unknown. 
# Create a complete file with all data included
registration_status = pd.read_csv("C:/Users/Orysya/Desktop/Civis_Analytics_Assessment/registration_status.csv")
individual_editted = pd.read_csv("C:/Users/Orysya/Desktop/Civis_Analytics_Assessment/individual_editted.csv")
merge = pd.merge(individual_editted, household, how='inner', on='hh_id')
all_data = pd.merge(merge, registration_status, how='inner', on="person_id")
all_data.to_csv("C:/Users/Orysya/Desktop/all_data.csv")

In [505]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
from patsy import dmatrices
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split
from sklearn import metrics
from sklearn.cross_validation import cross_val_score

In [382]:
# Create a dataframe with an intercept column
y, X = dmatrices('is_registered_democrat ~ race + voted_in_2012 + adult + hh_type + is_uban + is_owner_of_home + tercile_of_census_tract_income', all_data, return_type="dataframe")
# flatten y into 1-D array
y = np.ravel(y)

In [385]:
# evaluate the model by splitting it into training and testing data sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=1)
model2 = LogisticRegression()
model2.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr',
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0)

In [387]:
# predict class labels for the test data set
predicted = model2.predict(X_test)
predicted

array([ 0.,  0.,  0., ...,  0.,  0.,  0.])

In [388]:
# generate class probabilites
probs = model2.predict_proba(X_test)
probs
#The classifier is predicting 1 (is_registered_democrat) when probability in second column > 0.5

array([[ 0.9979509 ,  0.0020491 ],
       [ 0.99498741,  0.00501259],
       [ 0.80481737,  0.19518263],
       ..., 
       [ 0.9979509 ,  0.0020491 ],
       [ 0.74017677,  0.25982323],
       [ 0.87494207,  0.12505793]])

In [389]:
print metrics.accuracy_score(y_test, predicted)
print metrics.roc_auc_score(y_test, probs[:, 1])
# model classified is_registered_democrat = 1 with low accuracy, there is a need to choose the variables better

0.871994678324
0.703138582041


In [390]:
print metrics.confusion_matrix(y_test, predicted)
print metrics.classification_report(y_test, predicted)

[[9176    0]
 [1347    0]]
             precision    recall  f1-score   support

        0.0       0.87      1.00      0.93      9176
        1.0       0.00      0.00      0.00      1347

avg / total       0.76      0.87      0.81     10523

